<a href="https://colab.research.google.com/github/judem-21/Neural-Style-Transfer/blob/main/NST2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing all the necessary python libraries

import os
import torch
import torch.nn as nn

#importing all the optimisers such as Adam,momentum etc.
import torch.optim as optim

#importing the Python Image Library(PIL) to deal with loading of images and their file extension types
from PIL import Image

#to convert our images to vectors/pytorch tensors
import torchvision.transforms as transforms

#for importing the vgg 19 pretrained model
import torchvision.models as models

#to read,write and save images
from torchvision.utils import save_image

In [ ]:
#selecting the layers for the content loss and style loss computation
layers_content=[21,34]
layers_style=[0,5,10,19,28]

In [ ]:
#mounting google drive to access and store images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#creating a python class whose instance would be our network i.e the VGG 19 model
class VGG(nn.Module):

  def __init__(self):
    super(VGG,self).__init__()
    self.chosen_layers1=layers_content
    self.chosen_layers2=layers_style
    self.model=models.vgg19(pretrained=True).features

  def forward(self,x,chk):
    features_con=[]
    features_sty=[]

    #sending in the image through the model and storing activations of the respective layers of the content image style image and the generated image to compute the losses
    for layer_num,layer in enumerate(self.model):

      #passing the activations to the next layer
      x=layer(x)

      if layer_num in self.chosen_layers1:
        features_con.append(x)

      if layer_num in self.chosen_layers2:
        features_sty.append(x)
    if chk==0:
      return features_con,features_sty
    elif chk==1:
      return features_con
    else:
      return features_sty

In [ ]:
#setting our primary computing device to gpu if available
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
#setting image size to 256 by 256
image_size=256

loader=transforms.Compose(
    [
        transforms.Resize((image_size,image_size)),
        transforms.ToTensor(),
    ]
)

In [ ]:
#loading our image and transforming it to a pytorch tensor of the size that is acceptable by the model
def load_image(image_name):
  image = Image.open(image_name)

  #adding an extra dimension of the batchsize which is 1
  image = loader(image).unsqueeze(0)

  #migrating the tensors to the respective device for computation
  return image.to(device)

In [ ]:
#passing the image paths to the user defined function above in order to transform it to required tensors
content_img=load_image('/content/drive/MyDrive/Conv_images/NST/catim.jpg')
style_img=load_image('/content/drive/MyDrive/Conv_images/NST/conim.jpg')

#generating a random image i.e. initialising a 256 by 256 by 3 tensor with random pixel values
generated_img=torch.randn(content_img.shape,device=device,requires_grad=True)
#generated_img=content_img.clone().requires_grad_(True)

#setting the hyperparameters of the model
total_steps=10000 #number of times the generated image is going to be modified i.e. total number of iterations
learning_rate=0.7
alpha=1000
beta=1000

In [ ]:
generated_img.shape

In [ ]:
#storing the instance/VGG class object and locking its weights and biases using eval function
model=VGG().to(device).eval()

In [ ]:
#setting the optimizer as Adam for better results
optimizer=optim.Adam([generated_img],lr=learning_rate)

In [ ]:
stp=0

#starting the training process upto the set number of iterations which is usually about 1000
for step in range(total_steps):

  #passing the randomly generated image, the content image
  #and the style image through the instance of the VGG 19 class(user defined) to get the activations of the content and style layers
  generated_con,generated_sty=model(generated_img,0)
  content_features=model(content_img,1)
  style_features=model(style_img,2)
  #each of the above tensors have each element of theirs as a tensor of the activations of 1 layer
  #in other words each element of the above tensors represenrts one entire layer(all channels included)

  style_loss,content_loss,total_loss=0,0,0

  #computing content loss with the activations of the content layers for the generated image and content image
  for gen_c,contnt_feature in zip(generated_con,content_features):
    batch_size,channel,height,width=gen_c.shape
    content_loss+=torch.mean((gen_c-contnt_feature)**2)*0.5

    #computing style loss with the activations of the style layers for the generated image and style image
  for gen_s,style_feature in zip(generated_sty,style_features):
    batch_size,channel,height,width=gen_s.shape
    G=gen_s.view(channel,height*width).mm(gen_s.view(channel,height*width).t())
    #for style image
    A=style_feature.view(channel,height*width).mm(style_feature.view(channel,height*width).t())

    style_loss+=torch.mean((G-A)**2)

  #backpropagation process
  total_loss=alpha*content_loss + beta*style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  #updating and saving the image after every 200 iterations to track progress
  if step%200==0:
    stp+=1
    print('Step',stp,': Total loss:',total_loss,',Content loss:',content_loss,',Style loss:',style_loss)
    save_image(generated_img,'/content/drive/MyDrive/Conv_images/NST/generated.jpg')